In [2]:
!pip install requests pandas openpyxl

import requests
import time
import os
from urllib.parse import quote
import re
import pandas as pd

#token-based name extraction (stops at non-name words like "is/has..etc", blocks places like "college")
NAME_TOKEN = re.compile(r"^(?:[A-Z]\.|[A-Z][A-Za-z\-\']+\.?)$")

#Lists of words that serve as warning signs. Ensure numbers aren't inflated and no false positives
STOP_WORDS_LOWER = {
    "is", "was", "were", "are", "has", "have", "had",
    "offers", "offer", "specializes", "specializes", "completed",
    "interned", "trained", "certified", "currently", "often", "also",
    "a", "an", "the", "and", "of", "in", "at", "for", "with", "to", "from"
}
BAD_LASTWORDS = {  #things that are not a last name
    "College", "University", "Center", "Hospital", "System", "Clinic", "Medical",
    "Health", "Services", "Associates", "Group", "Institute", "Practice"
}
BAD_NAME_TERMS = {
    "college", "school", "university", "medical", "medicine",
    "hospital", "center", "centre", "clinic", "institute",
    "foundation", "program", "department", "services",
    "health", "system", "group", "associates"
}

def get_names(text):
    names = []
    #match "Dr." or "Dr" 
    for m in re.finditer(r"\bDr\.?\s+", text):
        i = m.end()
        #grab a short window after "Dr." and tokenize it
        window = text[i:i+80]
        tokens = re.findall(r"[A-Za-z\-\']+\.?", window)

        collected = []
        for t in tokens:
            #stop if hit normal sentence words
            if t.lower().strip(".") in STOP_WORDS_LOWER:
                break
            #keep only name-like tokens (capitalized words/initials)
            if not NAME_TOKEN.match(t):
                break
            collected.append(t.strip())
            if len(collected) >= 5:
                break
        #require at least first + last
        if len(collected) >= 2:
            last = collected[-1].strip(".")
            if last in BAD_LASTWORDS:
                continue
            full = "Dr. " + " ".join(collected).replace("  ", " ").strip()
            #reject institutions disguised as doctors (false positive)
            last_word = full.split()[-1].lower().strip(".,")
            if last_word in BAD_NAME_TERMS:
                continue
            names.append(full)
    #remove duplicates while keeping order
    names = list(dict.fromkeys(names))
    return names

#particles that can be part of a compound last name
LASTNAME_PARTICLES = {
    "da","de","del","della","der","di","la","le","van","von","st","st."
}
#helps prevent duplicate names from being counted towards number of surgeons in that procedure (keeps numbers from being inflated)
def key(dr_name: str):
    # remove "Dr." and extra spaces
    s = re.sub(r"^Dr\.\s*", "", dr_name).strip()
    s = re.sub(r"\s+", " ", s)
    #strip common credentials stuck onto the end (keeps names cleaner)
    s = re.sub(r"\b(MD|DO|DPM|DDS|DMD|PhD|FACS|FACFAS)\b\.?", "", s, flags=re.I).strip()
    s = re.sub(r"\s+", " ", s)
    parts = s.split()
    if not parts:
        return ("", "")
    #build last name (for compound last names like "Del Monte", "Van Dyke", etc.)
    last = parts[-1].lower()
    if len(parts) >= 2 and parts[-2].lower().rstrip(".") in LASTNAME_PARTICLES:
        last = (parts[-2] + " " + parts[-1]).lower()
    #if no clear first name leave first_initial blank
    first_initial = ""
    if len(parts) >= 3:
        first_initial = parts[0][0].lower()
    return (last, first_initial)
    
def dedupe_by_key(names):
    best_by_full = {}
    best_by_last = {}
    for n in names:
        last, fi = key(n)
        #if first initial is missing, dedupe by last name only
        if fi == "":
            if last not in best_by_last or len(n) > len(best_by_last[last]):
                best_by_last[last] = n
            continue
        k = (last, fi)
        if k not in best_by_full or len(n) > len(best_by_full[k]):
            best_by_full[k] = n
        if last not in best_by_last or len(n) > len(best_by_last[last]):
            best_by_last[last] = n
    #if a "last-only" entry matches a full entry's last name, keep the better one
    out = list(best_by_full.values())
    full_lasts = {k[0] for k in best_by_full.keys()}
    for last, n in best_by_last.items():
        if last not in full_lasts:
            out.append(n)
    return out

zipcodes = pd.read_excel("C:/Users/Harrison Ehee/Downloads/Python Projects/zipcodes.xlsx")
zipcodes.head()
#request headers
headers = {
    "accept": "text/x-component",
    "content-type": "text/plain;charset=UTF-8",
    "origin": "https://locator.treace.net",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "next-action": "f6fdafddf3397baac6cf9b5e5ca7ee6e2676372b",
}
#loop over all zipcodes + write results to excel
def fetch_surgeons_for_zip(zip_code, procedure, next_action, radius=25):
    #build a referer url 
    referer_url = f"https://locator.treace.net/?q={zip_code}&radius={radius}&procedure={quote(procedure)}"
    payload = f'[{{"query":"{zip_code}","radius":{radius},"filters":[],"procedure":"{procedure}","npi":"","surgeonName":""}}]'
    
    h = headers.copy()
    h["next-action"] = next_action
    h["referer"] = referer_url

    r = requests.post("https://locator.treace.net/", headers=h, data=payload, timeout=20)
    r.raise_for_status()

    names_raw = get_names(r.text)
    names = dedupe_by_key(names_raw)
    return names_raw, names

RADIUS = 25
NEXTACT_MIS = "f6fdafddf3397baac6cf9b5e5ca7ee6e2676372b"
PROCEDURE_MIS = "MISSurgeon"
NEXTACT_LAP = "f6fdafddf3397baac6cf9b5e5ca7ee6e2676372b"
PROCEDURE_LAP = "Lapiplasty Surgeon"

print("zipcode|MIS|Lap|Both|MIS_only|Lap_only")

#loop through every zipcode in your first column
rows = [] 
for i in range(len(zipcodes)):
    zip_code = str(zipcodes.iloc[i, 0]).split(".")[0].zfill(5)
    try:
        mis_raw, mis = fetch_surgeons_for_zip(zip_code, PROCEDURE_MIS, NEXTACT_MIS, radius=RADIUS)
        lap_raw, lap = fetch_surgeons_for_zip(zip_code, PROCEDURE_LAP, NEXTACT_LAP, radius=RADIUS)
        mis_set = set(mis)
        lap_set = set(lap)
        both = (mis_set & lap_set)
        mis_only = (mis_set - lap_set)
        lap_only = (lap_set - mis_set)
        rows.append({
            "Zipcode": zip_code,
            "MIS": len(mis_set),
            "Lapiplasty": len(lap_set),
            "Both": len(both),
            "MIS only": len(mis_only),
            "Lapiplasty only": len(lap_only),
        })
        print(f"{zip_code}|MIS={len(mis_set)}|Lap={len(lap_set)}|Both={len(both)}|MIS_only={len(mis_only)}|Lap_only={len(lap_only)}")
        time.sleep(0.4)
    except Exception as e:
        #still keep the row so output stays aligned to the input file order
        rows.append({
            "Zipcode": zip_code,
            "MIS": None,
            "Lapiplasty": None,
            "Both": None,
            "MIS only": None,
            "Lapiplasty only": None,
            "Error": str(e),
        })
        print(f"{zip_code}: ERROR -> {e}")

out_df = pd.DataFrame(rows)
#make a new file (with timestamp) each time code is run without overwriting previous file
base_dir = r"C:\Users\Harrison Ehee\Downloads\Python Projects"
timestamp = time.strftime("%Y%m%d_%H%M%S")
output_path = os.path.join(
    base_dir,
    f"treace_zipcode_surgeon_counts_{timestamp}.xlsx"
)

out_df.to_excel(output_path, index=False)
print("Saved:", output_path)


zipcode|MIS|Lap|Both|MIS_only|Lap_only
10022|MIS=7|Lap=21|Both=4|MIS_only=3|Lap_only=17
10024|MIS=6|Lap=21|Both=4|MIS_only=2|Lap_only=17
11215|MIS=11|Lap=19|Both=4|MIS_only=7|Lap_only=15
07078|MIS=11|Lap=21|Both=4|MIS_only=7|Lap_only=17
07302|MIS=11|Lap=21|Both=5|MIS_only=6|Lap_only=16
08540|MIS=5|Lap=5|Both=0|MIS_only=5|Lap_only=5
19103|MIS=3|Lap=10|Both=2|MIS_only=1|Lap_only=8
20007|MIS=4|Lap=7|Both=1|MIS_only=3|Lap_only=6
20815|MIS=4|Lap=7|Both=1|MIS_only=3|Lap_only=6
02116|MIS=0|Lap=3|Both=0|MIS_only=0|Lap_only=3
02445|MIS=0|Lap=5|Both=0|MIS_only=0|Lap_only=5
06830|MIS=3|Lap=8|Both=1|MIS_only=2|Lap_only=7
15213|MIS=0|Lap=2|Both=0|MIS_only=0|Lap_only=2
23226|MIS=0|Lap=0|Both=0|MIS_only=0|Lap_only=0
28207|MIS=0|Lap=3|Both=0|MIS_only=0|Lap_only=3
27608|MIS=0|Lap=4|Both=0|MIS_only=0|Lap_only=4
30309|MIS=7|Lap=8|Both=4|MIS_only=3|Lap_only=4
33146|MIS=10|Lap=6|Both=3|MIS_only=7|Lap_only=3
33139|MIS=10|Lap=6|Both=3|MIS_only=7|Lap_only=3
33480|MIS=4|Lap=4|Both=2|MIS_only=2|Lap_only=2
33308